In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Custom Layers 

In [2]:
!pip install d2l==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.2
    Uninstalling pyparsing-3.1.2:
      Successfully uninstalled pyparsing-3.1.2
  Attempting uninstall: numpy
    Found e

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [4]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [5]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [6]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

As an extra sanity check, we can send random data through the network and check that the mean is in fact 0. **Because we are dealing with floating point numbers, we may still see a very small nonzero number due to quantization.**

In [7]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.3132e-10, grad_fn=<MeanBackward0>)

In [8]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [9]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.2260, -1.6338, -0.8029],
        [-1.0574, -0.5111, -0.5638],
        [-0.3957, -0.3518,  0.2889],
        [-1.3225, -1.5120,  0.1144],
        [-0.8614,  0.5828, -0.9686]], requires_grad=True)

In [10]:
linear(torch.rand(2, 5))  # 前向传播

tensor([[0., 0., 0.],
        [0., 0., 0.]])

在PyTorch中，当你定义一个继承自`nn.Module`的类时，这个类会自动获得一些内置的方法和属性。其中一个重要的特性是，当你直接调用一个`nn.Module`的实例（就像调用一个函数一样）时，它会自动调用该实例的`forward`方法。

这是通过Python的特殊方法`__call__`实现的。`nn.Module`类重写了`__call__`方法，使得当你使用实例名加上括号和参数调用时（例如`linear(input)`），它实际上在内部调用了`forward`方法。

具体来说，当你执行`linear(torch.rand(2, 5))`时，以下是发生的事情：

1. **实例调用**：`linear`是`MyLinear`类的一个实例。
2. **自动调用 forward**：由于`MyLinear`继承自`nn.Module`，调用`linear(input)`时，PyTorch框架会自动调用`MyLinear`类中定义的`forward`方法。
3. **参数传递**：`torch.rand(2, 5)`生成的随机张量作为输入参数传递给`forward`方法。

这种设计使得模型的前向传播非常直观和简洁。你只需要定义`forward`方法，而不需要显式调用它，这简化了模型的使用和代码的可读性。

因此，`linear(torch.rand(2, 5))`这行代码之所以能计算前向传播，是因为PyTorch的`nn.Module`类的设计使得实例调用自动映射到`forward`方法的调用。


In [11]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [0.5741]])